Based on these posts: 
* https://medium.com/@chrisfotache/text-classification-in-python-pipelines-nlp-nltk-tf-idf-xgboost-and-more-b83451a327e0
* https://github.com/keisukeirie/Amazon_review_helpfulness_prediction
* https://stackabuse.com/text-classification-with-bert-tokenizer-and-tf-2-0-in-python/
* https://towardsdatascience.com/simple-bert-using-tensorflow-2-0-132cb19e9b22

In [1]:
!pip install -q boto3
!pip install -q scikit-learn==0.20.3

tensorflow 2.0.0 requires opt-einsum>=2.3.2, which is not installed.
tensorflow 2.0.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
awscli 1.18.11 has requirement botocore==1.15.11, but you'll have botocore 1.14.17 which is incompatible.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
tensorflow 2.0.0 requires opt-einsum>=2.3.2, which is not installed.
tensorflow 2.0.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
awscli 1.18.11 has requirement botocore==1.15.11, but you'll have botocore 1.14.17 which is incompatible.
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [3]:
!head ./data/amazon90000.csv

head: cannot open ‘./amazon20000.csv’ for reading: No such file or directory


In [4]:
df = pd.read_csv('amazon20000.csv')
df.shape

FileNotFoundError: [Errno 2] File b'amazon20000.csv' does not exist: b'amazon20000.csv'

In [ ]:
df.head(5)

# Drop any NaNs

In [ ]:
df.shape

In [ ]:
df.isna().values.any()

In [ ]:
df_cleaned_and_filtered = df.dropna()
df_cleaned_and_filtered = df_cleaned_and_filtered.reset_index()
df_cleaned_and_filtered.shape

In [ ]:
df_cleaned_and_filtered = df_dropna.query('helpful_votes > 0')
df_cleaned_and_filtered.shape

In [ ]:
df_cleaned_and_filtered.head(5)

In [ ]:
df_cleaned_and_filtered['pct_helpful_votes'] = df_cleaned_and_filtered['helpful_votes'] / df_cleaned_and_filtered['total_votes']
df_cleaned_and_filtered.shape

In [ ]:
df_cleaned_and_filtered.head(5)

In [ ]:
df_cleaned_and_filtered['is_helpful'] = df_cleaned_and_filtered['pct_helpful_votes'] > 0.75
df_cleaned_and_filtered.head(5)

In [ ]:
df_cleaned_and_filtered['is_positive_sentiment'] = (df_cleaned_and_filtered['star_rating'] >= 4).astype(int)
df_cleaned_and_filtered.head(5)

In [ ]:
import seaborn as sns

sns.countplot(x='is_positive_sentiment', data=df_cleaned_and_filtered)


## Balance the dataset

In [ ]:
from sklearn.utils import resample

is_negative_sentiment_df = df_cleaned_and_filtered.query('is_positive_sentiment == 0')
is_positive_sentiment_df = df_cleaned_and_filtered.query('is_positive_sentiment == 1')

is_positive_downsampled_df = resample(is_positive_sentiment_df,
                                      replace = False, # sample without replacement
                                      n_samples = len(is_negative_sentiment_df), # match minority n
                                      random_state = 27) # reproducible results

df_cleaned_and_filtered = pd.concat([is_negative_sentiment_df, is_positive_downsampled_df])


In [ ]:
sns.countplot(x='is_positive_sentiment', data=df_cleaned_and_filtered)


### BERT
MultiLabel Classification:  https://towardsdatascience.com/multi-label-classification-using-bert-roberta-xlnet-xlm-and-distilbert-with-simple-transformers-b3e0cda12ce5

BinaryClassification:  https://towardsdatascience.com/simple-transformers-introducing-the-easiest-bert-roberta-xlnet-and-xlm-library-58bf8c59b2a3

Install simpletransformers by Hugging Face folks

In [ ]:
!pip install simpletransformers==0.19.9

In [ ]:
!pip install tensorboardx==2.0

In [ ]:
!pip install torch==1.4.0

### Prepare the dataset for BERT.

By `simpletransformer` convention, the dataframe must have 2 columns:
* `text`
* `labels`

In [ ]:
df_bert = df_cleaned_and_filtered[['review_body', 'is_positive_sentiment']]
df_bert.columns = ['text', 'labels']
df_bert.head(5)

### Split the data into train and test.  

In [ ]:
from sklearn.model_selection import train_test_split

df_bert_train, df_bert_test = train_test_split(df_bert, test_size=0.10)

### Train the model

In [ ]:
from simpletransformers.classification import ClassificationModel

# Create a TransformerModel

args = {
#   'model_type':  'bert',
#   'model_name': 'bert-base-cased',
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',
   'fp16': False,
#   'fp16_opt_level': 'O1',
   'max_seq_length': 128,
   'train_batch_size': 8,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 1,
   'weight_decay': 0,
   'learning_rate': 3e-5,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,
   'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,
   'tensorboard_dir': 'runs',
   'overwrite_output_dir': True,
   'reprocess_input_data': False,
}

bert_model = ClassificationModel(model_type='bert', #roberta, etc.
                                 model_name='bert-base-cased',
                                 args=args,
                                 use_cuda=False)

bert_model.train_model(train_df=df_bert_train,
                       eval_df=df_bert_test,
                       show_running_loss=True)

In [ ]:
import sklearn

result, model_outputs, wrong_predictions = bert_model.eval_model(eval_df=df_bert_test, acc=sklearn.metrics.accuracy_score)

In [ ]:
predictions, raw_outputs = bert_model.predict(["""Very funny. A typical mid 50's comedy."""])
print('Predictions: {}'.format(predictions))
print('Raw outputs: {}'.format(raw_outputs))

In [ ]:
predictions, raw_outputs = bert_model.predict(["""That movie was absolutely awful."""])
print('Predictions: {}'.format(predictions))
print('Raw outputs: {}'.format(raw_outputs))